In [1]:
"""
Combine the non-prior background all-sky scan TS
with the GBM/fake healpix prior
to get the maximum TSs

This is a jupyter test.
The actual work is done with `02_get_background_TS_with_healpix.py`
"""

'\nCombine the non-prior background all-sky scan TS\nwith the GBM/fake healpix prior\nto get the maximum TSs\n\nThis is a jupyter test.\nThe actual work is done with `02_get_background_TS_with_healpix.py`\n'

In [1]:
import os
import sys
print("Python version: ", end=' ')
print(sys.version)

import numpy as np
import healpy as hp
# import histlite as hl
import csky as cy
import pandas as pd
from scipy import sparse

# import matplotlib.pyplot as plt
# from matplotlib import cm
# import matplotlib.colors as colors
############# comment out below two lines on clusters ##########
# %matplotlib inline
# %matplotlib notebook
################################################################
from glob import glob
timer = cy.timing.Timer()
time = timer.time

###### Local Import ######
sys.path.append('../../')
from greco_grb.scripts import SETTING
paths = SETTING.PATH()
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from greco_grb.scripts.utils import *

######### python multiprocessing #######
import multiprocessing as mp

Python version:  3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]

#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [2]:
import argparse

######################### CONFIGURE ARGUEMENTS #############################
# p = argparse.ArgumentParser(description="Background Trials",
#                             formatter_class=argparse.RawTextHelpFormatter)
# p.add_argument("--grb_name_idx_start", default=0, type=int, help="Starting index of one GRB")
# p.add_argument("--grb_name_idx_end", default=1, type=int, help="Ending index of one GRB")
# p.add_argument("--tw_in_second", default=10, type=int, help="Length of the time window in seconds")
# args = p.parse_args()
###########################################################################


### testing on jupyter ###

class args:
    grb_name_idx_start = 0    # [start, end)   [inclusive, exclusive)
    grb_name_idx_end = 1
    # grb_name = "GRB180423A"    # real healpix example
    # grb_name = "GRB190611B"    # fake healpix example
    tw_in_second = 500

##########################

In [3]:
print("\n===== Loading no-healpix df =====\n")
df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHealpix_2268.pkl")
# realHealpix: df.gbm & df.hasHealpix
# fakeHealpix: ~(df.gbm & df.hasHealpix)
# df.head()


===== Loading no-healpix df =====



In [4]:
files = glob(ANA_DIR + "/allsky_scan/no_prior_versatile/tw{}/GRB180423A_batchSize2500_batchIndex*_tw{}.npz".format(args.tw_in_second, args.tw_in_second))
files.sort(key=lambda x: int(x[x.find("batchIndex") + len("batchIndex"): x.find("_tw")]))


In [6]:
################ for testing  #############
# files = files[:4]
###########################################

In [5]:
with time("from idx {} to idx {}".format(args.grb_name_idx_start, args.grb_name_idx_end)):
    for grb_idx in range(args.grb_name_idx_start, args.grb_name_idx_end):
        grb_name = df.grb_name[grb_idx]
        print("\n===== Loading Prior {} =====\n".format(grb_name))
        probs = np.maximum(1e-15, np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(grb_name)))
        print("\n===== Done =====\n")
        print("\n===== Calculating MAXs =====\n")
        with time("{}_tw{}".format(grb_name, args.tw_in_second)):
            TSs = np.array([])
            with mp.Pool() as pool:
                tss = np.array(pool.starmap(getCombinedMaxTSfromFile, zip([probs]*len(files), files)))
                TSs = np.append(TSs, tss.flatten())
        print("\n===== Done =====\n")
    
        print("\n===== Saving... =====\n")
        sTSs = sparse.csr_matrix(TSs, dtype=float)
        outfilename = "{}_tw{}_NTrial{}.npz".format(grb_name, 
                                                     args.tw_in_second,
                                                    sTSs.shape[1])
        output_folder = cy.utils.ensure_dir(ANA_DIR+"/allsky_scan/with_prior_background/tw{}".format(args.tw_in_second))
        sparse.save_npz(output_folder+"/{}".format(outfilename) ,sTSs)
        print("\n===== Saved =====\n")

GRB200529B

0:04:53.206383 elapsed.

0:04:53.478357 elapsed.


In [45]:
"""
original implementation: not using multiprocessing
"""

'\noriginal implementation\n'

In [38]:
for grb_idx in range(args.grb_name_idx_start, args. grb_name_idx_end):
    grb_name = df.grb_name[grb_idx]
    print(grb_name)
    probs = np.maximum(1e-15, np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(grb_name)))
    
    with time("{}_tw{}".format(grb_name, args.tw_in_second)):
        TSs = []
        for i, file in enumerate(files):
            if i % (len(files)/50) == 0:
                print("Working on number of file: {}".format(i))
            scans = sparse.load_npz(file)
            ## sol0: 1 cpu
            for idx in range(scans.shape[0]):
                scan = scans[idx]
                TS = getCombinedMaxTS(probs, scan)
                TSs.append(TS)
            ## sol1: mp.Pool()
#             with mp.Pool() as pool:
#                 TSs.extend(pool.starmap(getCombinedMaxTS, zip([probs]*scans.shape[0], scans)))
            
                
    sTSs = sparse.csr_matrix(TSs, dtype=float)
    
    outfilename = "{}_tw{}_NTrial{}.npz".format(grb_name, 
                                                 args.tw_in_second,
                                                sTSs.shape[1])
    output_folder = cy.utils.ensure_dir(ANA_DIR+"/allsky_scan/with_prior_background/tw{}".format(args.tw_in_second))
#     sparse.save_npz(output_folder+"/{}".format(outfilename) ,sTSs)
 

GRB200529B
Working on number of file: 0
Working on number of file: 8
Working on number of file: 16
Working on number of file: 24
Working on number of file: 32
Working on number of file: 40
Working on number of file: 48
Working on number of file: 56
Working on number of file: 64
Working on number of file: 72
Working on number of file: 80
Working on number of file: 88
Working on number of file: 96
Working on number of file: 104
Working on number of file: 112
Working on number of file: 120
Working on number of file: 128
Working on number of file: 136
Working on number of file: 144
Working on number of file: 152
Working on number of file: 160
Working on number of file: 168
Working on number of file: 176
Working on number of file: 184
Working on number of file: 192
Working on number of file: 200
Working on number of file: 208
Working on number of file: 216
Working on number of file: 224
Working on number of file: 232
Working on number of file: 240
Working on number of file: 248
Working on n

In [39]:
# 21cpu*min for 1 GRB for 1 TW
21 * 2268 * 6

# using 4 cpus: 25min for 1 GRB for 1 TW.....

285768

In [40]:
mp.cpu_count()

24